In [7]:
import requests
import json

# Load configuration from JSON file
with open('credentials.json', 'r') as config_file:
    config = json.load(config_file)

read_api_key = config['read_api']['api_token']
write_api_key =  config['write_api']['api_token']

In [8]:
import whisperx
import gc

device = "cpu"
audio_file = "F&H/f&h_english.wav"
batch_size = 4 # reduce if low on GPU mem
compute_type = "int8" # change to "int8" if low on GPU mem (may reduce accuracy)

In [10]:
from openai import OpenAI

client = OpenAI()

audio_file_openai = open("F&H/f&h_english.wav", "rb")
transcript2 = client.audio.transcriptions.create(
  file=audio_file_openai,
  model="whisper-1",
  response_format="verbose_json",
  timestamp_granularities=["word","segment"]
)

print(transcript2)
print(transcript2.language)
print(transcript2.duration)
print(transcript2.segments)
print(transcript2.words)

Transcription(text="Hello Farhan how are you? I'm fine how about you? I'm good let's talk about Fiverr success I'm recently created account on Fiverr but I have not success and can't get any order from there so can you please tell me some guidance or some tips from how then I can succeed from there. Okay before proceeding further I want to ask you what niche are you working in? Currently I am providing my services on in data sciences and artificial intelligence. Wow that's good okay so you are not getting impressions or orders? Both in impression I also can't get a more impression and the success rate is also not good. Okay so I would advise you that you should go for sharing your Fiverr profile in your whatsapp group Facebook groups that is the first step in the in our next call I will tell you about the second tip that that is all for today. Is it feasible for me or I can attempt some more techniques like? No for now just focus on this technique okay and then I will tell you the more

In [12]:
for i in range(len(transcript2.words)):
    print(transcript2.words[i])

{'word': 'Hello', 'start': 0.46000000834465027, 'end': 1.1399999856948853}
{'word': 'Farhan', 'start': 1.1399999856948853, 'end': 1.7400000095367432}
{'word': 'how', 'start': 1.7400000095367432, 'end': 1.8600000143051147}
{'word': 'are', 'start': 1.8600000143051147, 'end': 2.0199999809265137}
{'word': 'you', 'start': 2.0199999809265137, 'end': 2.359999895095825}
{'word': "I'm", 'start': 2.759999990463257, 'end': 2.940000057220459}
{'word': 'fine', 'start': 2.940000057220459, 'end': 3.140000104904175}
{'word': 'how', 'start': 3.140000104904175, 'end': 3.4000000953674316}
{'word': 'about', 'start': 3.4000000953674316, 'end': 4.139999866485596}
{'word': 'you', 'start': 4.139999866485596, 'end': 4.800000190734863}
{'word': "I'm", 'start': 5.28000020980835, 'end': 5.380000114440918}
{'word': 'good', 'start': 5.380000114440918, 'end': 5.599999904632568}
{'word': "let's", 'start': 5.599999904632568, 'end': 6.739999771118164}
{'word': 'talk', 'start': 6.739999771118164, 'end': 6.94000005722045

In [10]:
filtered_segments = []

for i in range(len(transcript2.segments)):
    data = transcript2.segments[i]  # Access the current segment within the loop

    # Extract only 'text', 'start', and 'end'
    filtered_data = {'text': data['text'], 'start': data['start'], 'end': data['end']}
    
    # Append the filtered data to the list
    filtered_segments.append(filtered_data)

# Print the filtered segments
for segment in filtered_segments:
    print(segment)

print("-------------------------------------------------------------------------------------")
print(filtered_segments)

{'text': " Hello Farhan how are you? I'm fine how about you? I'm good let's talk about", 'start': 0.0, 'end': 7.199999809265137}
{'text': " Fiverr success I'm recently created account on Fiverr but I have not", 'start': 7.199999809265137, 'end': 12.319999694824219}
{'text': " success and can't get any order from there so can you please tell me some", 'start': 12.319999694824219, 'end': 17.68000030517578}
{'text': ' guidance or some tips from how then I can succeed from there. Okay before', 'start': 17.68000030517578, 'end': 23.520000457763672}
{'text': ' proceeding further I want to ask you what niche are you working in?', 'start': 23.520000457763672, 'end': 29.0}
{'text': ' Currently I am providing my services in data sciences and artificial intelligence. Wow', 'start': 29.0, 'end': 32.91999816894531}
{'text': " that's good okay so you are not getting impressions or orders? Both in impression", 'start': 32.91999816894531, 'end': 39.400001525878906}
{'text': " I also can't get a more i

In [11]:
audio = whisperx.load_audio(audio_file)

In [ ]:
print(audio_file)

In [15]:
# Mapping of language names to ISO 639-1 codes
language_mapping = {
    "english": "en",
    # Add more mappings as needed
}

result1 = {}
result1["segments"] = language_mapping.get(transcript2.language.lower(), transcript2.language)
print(result1["segments"])

en


In [16]:
# 2. Align whisper output
model_a, metadata = whisperx.load_align_model(language_code=result1["segments"], device=device)
result2 = whisperx.align(filtered_segments, model_a, metadata, audio, device, return_char_alignments=False)

print(result2["segments"]) # after alignment

[{'start': 0.923, 'end': 2.106, 'text': ' Hello Farhan how are you?', 'words': [{'word': 'Hello', 'start': 0.923, 'end': 1.284, 'score': 0.55}, {'word': 'Farhan', 'start': 1.564, 'end': 1.745, 'score': 0.304}, {'word': 'how', 'start': 1.765, 'end': 1.845, 'score': 0.811}, {'word': 'are', 'start': 1.905, 'end': 1.965, 'score': 0.631}, {'word': 'you?', 'start': 2.006, 'end': 2.106, 'score': 0.871}]}, {'start': 2.788, 'end': 4.292, 'text': "I'm fine how about you?", 'words': [{'word': "I'm", 'start': 2.788, 'end': 2.928, 'score': 0.822}, {'word': 'fine', 'start': 2.988, 'end': 3.269, 'score': 0.742}, {'word': 'how', 'start': 3.389, 'end': 3.931, 'score': 0.675}, {'word': 'about', 'start': 3.951, 'end': 4.152, 'score': 0.359}, {'word': 'you?', 'start': 4.172, 'end': 4.292, 'score': 0.869}]}, {'start': 5.194, 'end': 7.16, 'text': "I'm good let's talk about", 'words': [{'word': "I'm", 'start': 5.194, 'end': 5.355, 'score': 0.795}, {'word': 'good', 'start': 5.375, 'end': 5.575, 'score': 0.976

In [17]:
# 3. Assign speaker labels
diarize_model = whisperx.DiarizationPipeline(use_auth_token=write_api_key, device=device)

# add min/max number of speakers if known
diarize_segments = diarize_model(audio)
# diarize_model(audio, min_speakers=min_speakers, max_speakers=max_speakers)

result3 = whisperx.assign_word_speakers(diarize_segments, result2)
print(diarize_segments)
print(result3["segments"]) # segments are now assigned speaker IDs

                              segment label     speaker      start        end  \
0   [ 00:00:00.874 -->  00:00:22.419]     A  SPEAKER_00   0.874363  22.419355   
1   [ 00:00:22.707 -->  00:00:29.176]     B  SPEAKER_01  22.707980  29.176570   
2   [ 00:00:28.972 -->  00:00:32.707]     C  SPEAKER_00  28.972835  32.707980   
3   [ 00:00:32.707 -->  00:00:38.005]     D  SPEAKER_01  32.707980  38.005093   
4   [ 00:00:38.005 -->  00:00:43.743]     E  SPEAKER_00  38.005093  43.743633   
5   [ 00:00:44.117 -->  00:00:44.337]     F  SPEAKER_01  44.117148  44.337861   
6   [ 00:00:44.490 -->  00:01:02.334]     G  SPEAKER_01  44.490662  62.334465   
7   [ 00:01:00.229 -->  00:01:00.823]     H  SPEAKER_00  60.229202  60.823430   
8   [ 00:01:01.162 -->  00:01:01.230]     I  SPEAKER_00  61.162988  61.230900   
9   [ 00:01:01.332 -->  00:01:01.349]     J  SPEAKER_00  61.332767  61.349745   
10  [ 00:01:01.604 -->  00:01:08.718]     K  SPEAKER_00  61.604414  68.718166   
11  [ 00:01:08.242 -->  00:0

In [18]:
def extract_data(json_data):
    for segment in json_data:
       output = print(segment['speaker'] + ": " + segment['text'])
    return output

extract_data(result3['segments'])

SPEAKER_00:  Hello Farhan how are you?
SPEAKER_00: I'm fine how about you?
SPEAKER_00: I'm good let's talk about
SPEAKER_00:  Fiverr success I'm recently created account on Fiverr but I have not
SPEAKER_00:  success and can't get any order from there so can you please tell me some
SPEAKER_00:  guidance or some tips from how then I can succeed from there.
SPEAKER_01: Okay before
SPEAKER_01:  proceeding further I want to ask you what niche are you working in?
SPEAKER_00:  Currently I am providing my services in data sciences and artificial intelligence.
SPEAKER_01: Wow
SPEAKER_01:  that's good okay so you are not getting impressions or orders?
SPEAKER_00: Both in impression
SPEAKER_00:  I also can't get a more impression and the success rate is also not good.
SPEAKER_01: Okay
SPEAKER_01:  so I would advise you that you should go for sharing your Fiverr profile in
SPEAKER_01:  your whatsapp group Facebook groups that is the first step in the in our next
SPEAKER_01:  call I will tell you a

## *Final Script*

In [19]:
import requests
import json

# Load configuration from JSON file
with open('credentials.json', 'r') as config_file:
    config = json.load(config_file)

read_api_key = config['read_api']['api_token']
write_api_key =  config['write_api']['api_token']

import whisperx
import gc

device = "cpu"
audio_file_latest = "hassan/four.wav"
batch_size = 4 # reduce if low on GPU mem
compute_type = "int8" # change to "int8" if low on GPU mem (may reduce accuracy)

from openai import OpenAI

client = OpenAI()

audio_file_openai = open(audio_file_latest, "rb")
transcript_latest = client.audio.transcriptions.create(
  file=audio_file_openai,
  model="whisper-1",
  response_format="verbose_json",
  timestamp_granularities=["segment"]
)

filtered_segments_latest = []

for i in range(len(transcript_latest.segments)):
    data_latest = transcript_latest.segments[i]  # Access the current segment within the loop

    # Extract only 'text', 'start', and 'end'
    filtered_data_latest = {'text': data_latest['text'], 'start': data_latest['start'], 'end': data_latest['end']}
    
    # Append the filtered data to the list
    filtered_segments_latest.append(filtered_data_latest)

# Print the filtered segments
for segment in filtered_segments_latest:
    print(segment)

print("-------------------------------------------------------------------------------------")
print(filtered_segments_latest)

audio_latest = whisperx.load_audio(audio_file_latest)

# Mapping of language names to ISO 639-1 codes
language_mapping = {
    "afrikaans": "af",
    "arabic": "ar",
    "armenian": "hy",
    "azerbaijani": "az",
    "belarusian": "be",
    "bosnian": "bs",
    "bulgarian": "bg",
    "catalan": "ca",
    "chinese": "zh",
    "croatian": "hr",
    "czech": "cs",
    "danish": "da",
    "dutch": "nl",
    "english": "en",
    "estonian": "et",
    "finnish": "fi",
    "french": "fr",
    "galician": "gl",
    "german": "de",
    "greek": "el",
    "hebrew": "he",
    "hindi": "hi",
    "hungarian": "hu",
    "icelandic": "is",
    "indonesian": "id",
    "italian": "it",
    "japanese": "ja",
    "kannada": "kn",
    "kazakh": "kk",
    "korean": "ko",
    "latvian": "lv",
    "lithuanian": "lt",
    "macedonian": "mk",
    "malay": "ms",
    "marathi": "mr",
    "maori": "mi",
    "nepali": "ne",
    "norwegian": "no",
    "persian": "fa",
    "polish": "pl",
    "portuguese": "pt",
    "romanian": "ro",
    "russian": "ru",
    "serbian": "sr",
    "slovak": "sk",
    "slovenian": "sl",
    "spanish": "es",
    "swahili": "sw",
    "swedish": "sv",
    "tagalog": "tl",
    "tamil": "ta",
    "thai": "th",
    "turkish": "tr",
    "ukrainian": "uk",
    "urdu": "ur",
    "vietnamese": "vi",
    "welsh": "cy",
    # Add more mappings as needed
}
result1_latest = {}
result1_latest["segments"] = language_mapping.get(transcript_latest.language.lower(), transcript_latest.language)
print(result1_latest["segments"])


# 2. Align whisper output
model_a, metadata = whisperx.load_align_model(language_code=result1_latest["segments"], device=device)
result2_latest = whisperx.align(filtered_segments_latest, model_a, metadata, audio_latest, device, return_char_alignments=False)

print(result2_latest["segments"]) # after alignment


# 3. Assign speaker labels
diarize_model_latest = whisperx.DiarizationPipeline(use_auth_token=write_api_key, device=device)

# add min/max number of speakers if known
diarize_segments_latest = diarize_model_latest(audio_latest)
diarize_model(audio_latest, min_speakers=2, max_speakers=5)

result3_latest = whisperx.assign_word_speakers(diarize_segments_latest, result2_latest)
print(diarize_segments_latest)
print(result3_latest["segments"]) # segments are now assigned speaker IDs

{'text': ' I want to know why Loki let us take him.', 'start': 0.0, 'end': 2.0}
{'text': " He's not leading an army from here.", 'start': 2.0, 'end': 3.5}
{'text': " I don't think we should be focusing on Loki.", 'start': 3.5, 'end': 5.0}
{'text': " That guy's brain is a bag full of cats.", 'start': 5.0, 'end': 7.0}
{'text': ' You could smell crazy on him.', 'start': 7.0, 'end': 8.5}
{'text': " I've care how you speak.", 'start': 8.5, 'end': 11.0}
{'text': ' Loki is beyond reason, but he is of Asgard.', 'start': 11.0, 'end': 13.5}
{'text': ' And he is my brother.', 'start': 13.5, 'end': 15.0}
{'text': ' He killed 80 people in two days.', 'start': 15.0, 'end': 17.5}
{'text': " He's adopted?", 'start': 17.5, 'end': 19.0}
-------------------------------------------------------------------------------------
[{'text': ' I want to know why Loki let us take him.', 'start': 0.0, 'end': 2.0}, {'text': " He's not leading an army from here.", 'start': 2.0, 'end': 3.5}, {'text': " I don't think we

## With min/max speaker parameters

In [35]:
import requests
import json

# Load configuration from JSON file
with open('credentials.json', 'r') as config_file:
    config = json.load(config_file)

read_api_key = config['read_api']['api_token']
write_api_key =  config['write_api']['api_token']

import whisperx
import gc

device = "cpu"
audio_file_latest = "hassan/urdu audio.mp3"
batch_size = 4 # reduce if low on GPU mem
compute_type = "int8" # change to "int8" if low on GPU mem (may reduce accuracy)

from openai import OpenAI

client = OpenAI()

audio_file_openai = open(audio_file_latest, "rb")
transcript_latest = client.audio.transcriptions.create(
  file=audio_file_openai,
  model="whisper-1",
  response_format="verbose_json",
  timestamp_granularities=["segment"]
)

filtered_segments_latest = []

for i in range(len(transcript_latest.segments)):
    data_latest = transcript_latest.segments[i]  # Access the current segment within the loop

    # Extract only 'text', 'start', and 'end'
    filtered_data_latest = {'text': data_latest['text'], 'start': data_latest['start'], 'end': data_latest['end']}
    
    # Append the filtered data to the list
    filtered_segments_latest.append(filtered_data_latest)

# Print the filtered segments
for segment in filtered_segments_latest:
    print(segment)

print("-------------------------------------------------------------------------------------")
print(filtered_segments_latest)

audio_latest = whisperx.load_audio(audio_file_latest)

# Mapping of language names to ISO 639-1 codes
language_mapping = {
    "afrikaans": "af",
    "arabic": "ar",
    "armenian": "hy",
    "azerbaijani": "az",
    "belarusian": "be",
    "bosnian": "bs",
    "bulgarian": "bg",
    "catalan": "ca",
    "chinese": "zh",
    "croatian": "hr",
    "czech": "cs",
    "danish": "da",
    "dutch": "nl",
    "english": "en",
    "estonian": "et",
    "finnish": "fi",
    "french": "fr",
    "galician": "gl",
    "german": "de",
    "greek": "el",
    "hebrew": "he",
    "hindi": "hi",
    "hungarian": "hu",
    "icelandic": "is",
    "indonesian": "id",
    "italian": "it",
    "japanese": "ja",
    "kannada": "kn",
    "kazakh": "kk",
    "korean": "ko",
    "latvian": "lv",
    "lithuanian": "lt",
    "macedonian": "mk",
    "malay": "ms",
    "marathi": "mr",
    "maori": "mi",
    "nepali": "ne",
    "norwegian": "no",
    "persian": "fa",
    "polish": "pl",
    "portuguese": "pt",
    "romanian": "ro",
    "russian": "ru",
    "serbian": "sr",
    "slovak": "sk",
    "slovenian": "sl",
    "spanish": "es",
    "swahili": "sw",
    "swedish": "sv",
    "tagalog": "tl",
    "tamil": "ta",
    "thai": "th",
    "turkish": "tr",
    "ukrainian": "uk",
    "urdu": "ur",
    "vietnamese": "vi",
    "welsh": "cy",
    # Add more mappings as needed
}
result1_latest = {}
result1_latest["segments"] = language_mapping.get(transcript_latest.language.lower(), transcript_latest.language)
print(result1_latest["segments"])


# 2. Align whisper output
model_a, metadata = whisperx.load_align_model(language_code=result1_latest["segments"], device=device)
result2_latest = whisperx.align(filtered_segments_latest, model_a, metadata, audio_latest, device, return_char_alignments=False)

print(result2_latest["segments"]) # after alignment


# 3. Assign speaker labels
diarize_model_latest = whisperx.DiarizationPipeline(use_auth_token=write_api_key, device=device)

# add min/max number of speakers if known
diarize_segments_latest = diarize_model_latest(audio_latest)
# diarize_model(audio_latest, min_speakers=2, max_speakers=5)

result3_latest = whisperx.assign_word_speakers(diarize_segments_latest, result2_latest)
print(diarize_segments_latest)
print(result3_latest["segments"]) # segments are now assigned speaker IDs

{'text': ' السلام علیکم میں حبیبینک کا نمائندہ ہوں ہم آپ کو کارڈ لینے کی تجویز کریں گے کیا آپ انڈسٹر ہیں؟', 'start': 0.0, 'end': 7.0}
{'text': ' وعلیکم سلام جی ضرور پلیس بتائیے آپ کے پاس کون سے کریڈ کارڈز اویلیبل ہیں؟', 'start': 7.0, 'end': 12.0}
{'text': ' ہمارے کریڈ کارڈ میں کافی آسانیاں ہیں رقم نکروانے سے لے کر اور بھی بہت سی ایسی خاصوصیت ہیں جو دوسرے بینک کے کارڈ سے ہمیں الگ بناتی ہیں', 'start': 12.0, 'end': 20.0}
{'text': ' اچھا یہ تو بہت اچھی بات ہے پلیس کیا آپ ڈیٹیئر شیئر کر سکتے ہیں تاکہ میں اس کو دیکھ سکوں؟', 'start': 20.0, 'end': 25.0}
{'text': ' ٹھیک ہے آپ ہماری ویب سائیٹ پر فارم فل کر دیں اس کے بعد ساری ڈیٹیئر آپ کو مل جائیں گے بہت شکریہ', 'start': 25.0, 'end': 31.0}
{'text': ' تو کیا میں آپ کو پرسنلی میسج کر سکتا ہوں؟', 'start': 31.0, 'end': 34.0}
{'text': ' نہیں بالکل نہیں آپ ہماری ویب سائیٹ پر جا کے فارم فل کریں', 'start': 34.0, 'end': 38.0}
{'text': ' ٹھیک ہے میں ویب سائیٹ پر جا کے کرتا ہوں تینکیو', 'start': 38.0, 'end': 41.0}
-------------------------------------------

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of the model checkpoint at kingabzpro/wav2vec2-large-xls-r-300m-Urdu were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at kingabzpro/wav2vec2-large-xls-r-300m-Urdu and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.p

[{'start': 0.0, 'end': 7.0, 'text': ' السلام علیکم میں حبیبینک کا نمائندہ ہوں ہم آپ کو کارڈ لینے کی تجویز کریں گے کیا آپ انڈسٹر ہیں؟', 'words': [{'word': 'السلام', 'start': 0.0, 'end': 1.324, 'score': 0.164}, {'word': 'علیکم', 'start': 1.464, 'end': 1.825, 'score': 0.262}, {'word': 'میں', 'start': 1.926, 'end': 2.046, 'score': 0.403}, {'word': 'حبیبینک', 'start': 2.106, 'end': 2.668, 'score': 0.335}, {'word': 'کا', 'start': 2.688, 'end': 2.768, 'score': 0.603}, {'word': 'نمائندہ', 'start': 2.808, 'end': 3.269, 'score': 0.233}, {'word': 'ہوں', 'start': 3.289, 'end': 3.47, 'score': 0.083}, {'word': 'ہم', 'start': 3.59, 'end': 3.731, 'score': 0.567}, {'word': 'آپ', 'start': 3.751, 'end': 3.851, 'score': 0.006}, {'word': 'کو', 'start': 3.891, 'end': 3.971, 'score': 0.254}, {'word': 'کارڈ', 'start': 4.032, 'end': 4.252, 'score': 0.064}, {'word': 'لینے', 'start': 4.272, 'end': 4.473, 'score': 0.203}, {'word': 'کی', 'start': 4.493, 'end': 4.573, 'score': 0.279}, {'word': 'تجویز', 'start': 4.6

In [36]:
result2_latest

{'segments': [{'start': 0.0,
   'end': 7.0,
   'text': ' السلام علیکم میں حبیبینک کا نمائندہ ہوں ہم آپ کو کارڈ لینے کی تجویز کریں گے کیا آپ انڈسٹر ہیں؟',
   'words': [{'word': 'السلام',
     'start': 0.0,
     'end': 1.324,
     'score': 0.164,
     'speaker': 'SPEAKER_00'},
    {'word': 'علیکم',
     'start': 1.464,
     'end': 1.825,
     'score': 0.262,
     'speaker': 'SPEAKER_00'},
    {'word': 'میں',
     'start': 1.926,
     'end': 2.046,
     'score': 0.403,
     'speaker': 'SPEAKER_00'},
    {'word': 'حبیبینک',
     'start': 2.106,
     'end': 2.668,
     'score': 0.335,
     'speaker': 'SPEAKER_00'},
    {'word': 'کا',
     'start': 2.688,
     'end': 2.768,
     'score': 0.603,
     'speaker': 'SPEAKER_00'},
    {'word': 'نمائندہ',
     'start': 2.808,
     'end': 3.269,
     'score': 0.233,
     'speaker': 'SPEAKER_00'},
    {'word': 'ہوں',
     'start': 3.289,
     'end': 3.47,
     'score': 0.083,
     'speaker': 'SPEAKER_00'},
    {'word': 'ہم',
     'start': 3.59,
     

In [37]:
def extract_data(json_data):
    for segment in json_data:
       output = print(segment['speaker'] + ": " + segment['text'])
    return output

extract_data(result3_latest['segments'])

SPEAKER_00:  السلام علیکم میں حبیبینک کا نمائندہ ہوں ہم آپ کو کارڈ لینے کی تجویز کریں گے کیا آپ انڈسٹر ہیں؟
SPEAKER_01:  وعلیکم سلام جی ضرور پلیس بتائیے آپ کے پاس کون سے کریڈ کارڈز اویلیبل ہیں؟
SPEAKER_00:  ہمارے کریڈ کارڈ میں کافی آسانیاں ہیں رقم نکروانے سے لے کر اور بھی بہت سی ایسی خاصوصیت ہیں جو دوسرے بینک کے کارڈ سے ہمیں الگ بناتی ہیں
SPEAKER_03:  اچھا یہ تو بہت اچھی بات ہے پلیس کیا آپ ڈیٹیئر شیئر کر سکتے ہیں تاکہ میں اس کو دیکھ سکوں؟
SPEAKER_00:  ٹھیک ہے آپ ہماری ویب سائیٹ پر فارم فل کر دیں اس کے بعد ساری ڈیٹیئر آپ کو مل جائیں گے بہت شکریہ
SPEAKER_02:  تو کیا میں آپ کو پرسنلی میسج کر سکتا ہوں؟
SPEAKER_00:  نہیں بالکل نہیں آپ ہماری ویب سائیٹ پر جا کے فارم فل کریں
SPEAKER_02:  ٹھیک ہے میں ویب سائیٹ پر جا کے کرتا ہوں تینکیو


## Min Speaker == 0 & Max Speaker == 1

In [22]:
import requests
import json

# Load configuration from JSON file
with open('credentials.json', 'r') as config_file:
    config = json.load(config_file)

read_api_key = config['read_api']['api_token']
write_api_key =  config['write_api']['api_token']

import whisperx
import gc

device = "cpu"
audio_file_latest = "hassan/urdu audio.mp3"
batch_size = 4 # reduce if low on GPU mem
compute_type = "int8" # change to "int8" if low on GPU mem (may reduce accuracy)

from openai import OpenAI

client = OpenAI()

audio_file_openai = open(audio_file_latest, "rb")
transcript_latest = client.audio.transcriptions.create(
  file=audio_file_openai,
  model="whisper-1",
  response_format="verbose_json",
  timestamp_granularities=["segment"]
)

filtered_segments_latest = []

for i in range(len(transcript_latest.segments)):
    data_latest = transcript_latest.segments[i]  # Access the current segment within the loop

    # Extract only 'text', 'start', and 'end'
    filtered_data_latest = {'text': data_latest['text'], 'start': data_latest['start'], 'end': data_latest['end']}
    
    # Append the filtered data to the list
    filtered_segments_latest.append(filtered_data_latest)

# Print the filtered segments
for segment in filtered_segments_latest:
    print(segment)

print("-------------------------------------------------------------------------------------")
print(filtered_segments_latest)

audio_latest = whisperx.load_audio(audio_file_latest)

# Mapping of language names to ISO 639-1 codes
language_mapping = {
    "afrikaans": "af",
    "arabic": "ar",
    "armenian": "hy",
    "azerbaijani": "az",
    "belarusian": "be",
    "bosnian": "bs",
    "bulgarian": "bg",
    "catalan": "ca",
    "chinese": "zh",
    "croatian": "hr",
    "czech": "cs",
    "danish": "da",
    "dutch": "nl",
    "english": "en",
    "estonian": "et",
    "finnish": "fi",
    "french": "fr",
    "galician": "gl",
    "german": "de",
    "greek": "el",
    "hebrew": "he",
    "hindi": "hi",
    "hungarian": "hu",
    "icelandic": "is",
    "indonesian": "id",
    "italian": "it",
    "japanese": "ja",
    "kannada": "kn",
    "kazakh": "kk",
    "korean": "ko",
    "latvian": "lv",
    "lithuanian": "lt",
    "macedonian": "mk",
    "malay": "ms",
    "marathi": "mr",
    "maori": "mi",
    "nepali": "ne",
    "norwegian": "no",
    "persian": "fa",
    "polish": "pl",
    "portuguese": "pt",
    "romanian": "ro",
    "russian": "ru",
    "serbian": "sr",
    "slovak": "sk",
    "slovenian": "sl",
    "spanish": "es",
    "swahili": "sw",
    "swedish": "sv",
    "tagalog": "tl",
    "tamil": "ta",
    "thai": "th",
    "turkish": "tr",
    "ukrainian": "uk",
    "urdu": "ur",
    "vietnamese": "vi",
    "welsh": "cy",
    # Add more mappings as needed
}
result1_latest = {}
result1_latest["segments"] = language_mapping.get(transcript_latest.language.lower(), transcript_latest.language)
print(result1_latest["segments"])


# 2. Align whisper output
model_a, metadata = whisperx.load_align_model(language_code=result1_latest["segments"], device=device)
result2_latest = whisperx.align(filtered_segments_latest, model_a, metadata, audio_latest, device, return_char_alignments=False)

print(result2_latest["segments"]) # after alignment


# 3. Assign speaker labels
diarize_model_latest = whisperx.DiarizationPipeline(use_auth_token=write_api_key, device=device)

# add min/max number of speakers if known
diarize_segments_latest = diarize_model_latest(audio_latest)
diarize_model_latest(audio_latest, min_speakers=0, max_speakers=1)

result3_latest = whisperx.assign_word_speakers(diarize_segments_latest, result2_latest)
print(diarize_segments_latest)
print(result3_latest["segments"]) # segments are now assigned speaker IDs

{'text': ' السلام علیکم میں حبیبینک کا نمائندہ ہوں ہم آپ کو کارڈ لینے کی تجویز کریں گے کیا آپ انڈسٹر ہیں؟', 'start': 0.0, 'end': 7.0}
{'text': ' وعلیکم سلام جی ضرور پلیس بتائیے آپ کے پاس کون سے کریڈ کارڈز اویلیبل ہیں؟', 'start': 7.0, 'end': 12.0}
{'text': ' ہمارے کریڈ کارڈ میں کافی آسانیاں ہیں رقم نکروانے سے لے کر اور بھی بہت سی ایسی خاصوصیت ہیں جو دوسرے بینک کے کارڈ سے ہمیں الگ بناتی ہیں', 'start': 12.0, 'end': 20.0}
{'text': ' اچھا یہ تو بہت اچھی بات ہے پلیس کیا آپ ڈیٹیئر شیئر کر سکتے ہیں تاکہ میں اس کو دیکھ سکوں؟', 'start': 20.0, 'end': 25.0}
{'text': ' ٹھیک ہے آپ ہماری ویب سائیٹ پر فارم فل کر دیں اس کے بعد ساری ڈیٹیئر آپ کو مل جائیں گے بہت شکریہ', 'start': 25.0, 'end': 31.0}
{'text': ' تو کیا میں آپ کو پرسنلی میسج کر سکتا ہوں؟', 'start': 31.0, 'end': 34.0}
{'text': ' نہیں بالکل نہیں آپ ہماری ویب سائیٹ پر جا کے فارم فل کریں', 'start': 34.0, 'end': 38.0}
{'text': ' ٹھیک ہے میں ویب سائیٹ پر جا کے کرتا ہوں تینکیو', 'start': 38.0, 'end': 41.0}
-------------------------------------------

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of the model checkpoint at kingabzpro/wav2vec2-large-xls-r-300m-Urdu were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at kingabzpro/wav2vec2-large-xls-r-300m-Urdu and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.p

[{'start': 0.0, 'end': 7.0, 'text': ' السلام علیکم میں حبیبینک کا نمائندہ ہوں ہم آپ کو کارڈ لینے کی تجویز کریں گے کیا آپ انڈسٹر ہیں؟', 'words': [{'word': 'السلام', 'start': 0.0, 'end': 1.324, 'score': 0.164}, {'word': 'علیکم', 'start': 1.464, 'end': 1.825, 'score': 0.262}, {'word': 'میں', 'start': 1.926, 'end': 2.046, 'score': 0.403}, {'word': 'حبیبینک', 'start': 2.106, 'end': 2.668, 'score': 0.335}, {'word': 'کا', 'start': 2.688, 'end': 2.768, 'score': 0.603}, {'word': 'نمائندہ', 'start': 2.808, 'end': 3.269, 'score': 0.233}, {'word': 'ہوں', 'start': 3.289, 'end': 3.47, 'score': 0.083}, {'word': 'ہم', 'start': 3.59, 'end': 3.731, 'score': 0.567}, {'word': 'آپ', 'start': 3.751, 'end': 3.851, 'score': 0.006}, {'word': 'کو', 'start': 3.891, 'end': 3.971, 'score': 0.254}, {'word': 'کارڈ', 'start': 4.032, 'end': 4.252, 'score': 0.064}, {'word': 'لینے', 'start': 4.272, 'end': 4.473, 'score': 0.203}, {'word': 'کی', 'start': 4.493, 'end': 4.573, 'score': 0.279}, {'word': 'تجویز', 'start': 4.6

In [24]:
transcript_latest.language

'urdu'

In [23]:
def extract_data(json_data):
    for segment in json_data:
       output = print(segment['speaker'] + ": " + segment['text'])
    return output

extract_data(result3_latest['segments'])

SPEAKER_00:  السلام علیکم میں حبیبینک کا نمائندہ ہوں ہم آپ کو کارڈ لینے کی تجویز کریں گے کیا آپ انڈسٹر ہیں؟
SPEAKER_01:  وعلیکم سلام جی ضرور پلیس بتائیے آپ کے پاس کون سے کریڈ کارڈز اویلیبل ہیں؟
SPEAKER_00:  ہمارے کریڈ کارڈ میں کافی آسانیاں ہیں رقم نکروانے سے لے کر اور بھی بہت سی ایسی خاصوصیت ہیں جو دوسرے بینک کے کارڈ سے ہمیں الگ بناتی ہیں
SPEAKER_03:  اچھا یہ تو بہت اچھی بات ہے پلیس کیا آپ ڈیٹیئر شیئر کر سکتے ہیں تاکہ میں اس کو دیکھ سکوں؟
SPEAKER_00:  ٹھیک ہے آپ ہماری ویب سائیٹ پر فارم فل کر دیں اس کے بعد ساری ڈیٹیئر آپ کو مل جائیں گے بہت شکریہ
SPEAKER_02:  تو کیا میں آپ کو پرسنلی میسج کر سکتا ہوں؟
SPEAKER_00:  نہیں بالکل نہیں آپ ہماری ویب سائیٹ پر جا کے فارم فل کریں
SPEAKER_02:  ٹھیک ہے میں ویب سائیٹ پر جا کے کرتا ہوں تینکیو


## Here is the New work:

In [39]:
import requests
import json

# Load configuration from JSON file
with open('credentials.json', 'r') as config_file:
    config = json.load(config_file)

read_api_key = config['read_api']['api_token']
write_api_key =  config['write_api']['api_token']

import whisperx
import gc

device = "cpu"
audio_file_latest = "F&H/f&h_english.wav"
batch_size = 4 # reduce if low on GPU mem
compute_type = "int8" # change to "int8" if low on GPU mem (may reduce accuracy)

from openai import OpenAI

client = OpenAI()

audio_file_openai = open(audio_file_latest, "rb")
transcript_latest = client.audio.transcriptions.create(
  file=audio_file_openai,
  model="whisper-1",
  response_format="verbose_json",
  timestamp_granularities=["word","segment"]
)

filtered_segments_latest = []

for i in range(len(transcript_latest.segments)):
    data_latest = transcript_latest.segments[i]  # Access the current segment within the loop

    # Extract only 'text', 'start', and 'end'
    filtered_data_latest = {'text': data_latest['text'], 'start': data_latest['start'], 'end': data_latest['end']}
    
    # Append the filtered data to the list
    filtered_segments_latest.append(filtered_data_latest)

print("-------------------------------------------------------------------------------------")

# Rearrange the sequence of each dictionary element
corrected_data = [{'start': item['start'], 'end': item['end'], 'text': item['text']} for item in filtered_segments_latest]

# Printing the corrected data
for item in corrected_data:
    print(item)

print("-------------------------------------------------------------------------------------")

# Combine data from wrong_data and words
combined_data = []
for idx, data in enumerate(filtered_segments_latest):
    data_copy = data.copy()
    data_copy['words'] = []
    for word in transcript_latest.words:
        if data['start'] <= word['start'] <= data['end']:
            data_copy['words'].append(word)
    combined_data.append(data_copy)

print("-------------------------------------------------------------------------------------")
print(filtered_segments_latest)

audio_latest = whisperx.load_audio(audio_file_latest)

# Mapping of language names to ISO 639-1 codes
language_mapping = { "afrikaans": "af", "arabic": "ar", "armenian": "hy", "azerbaijani": "az", "belarusian": "be", "bosnian": "bs", 
                    "bulgarian": "bg", "catalan": "ca", "chinese": "zh", "croatian": "hr", "czech": "cs", "danish": "da",
                    "dutch": "nl", "english": "en", "estonian": "et", "finnish": "fi", "french": "fr", "galician": "gl", "german": "de",
                    "greek": "el", "hebrew": "he", "hindi": "ur", "hungarian": "hu", "icelandic": "is", "indonesian": "id", "italian": "it", 
                    "japanese": "ja", "kannada": "kn", "kazakh": "kk", "korean": "ko", "latvian": "lv", "lithuanian": "lt", "macedonian": "mk", 
                    "malay": "ms", "marathi": "mr",  "maori": "mi",  "nepali": "ne",  "norwegian": "no",  "persian": "fa",  "polish": "pl", 
                    "portuguese": "pt", "romanian": "ro", "russian": "ru", "serbian": "sr", "slovak": "sk", "slovenian": "sl", "spanish": "es", 
                    "swahili": "sw", "swedish": "sv", "tagalog": "tl", "tamil": "ta", "thai": "th", "turkish": "tr", "ukrainian": "uk", 
                    "urdu": "ur", "vietnamese": "vi", "welsh": "cy",
                    # Add more mappings as needed
                }
result1_latest = {}
result1_latest["segments"] = language_mapping.get(transcript_latest.language.lower(), transcript_latest.language)
print( "Combined Data: ", combined_data)


# 2. Align whisper output
model_a, metadata = whisperx.load_align_model(language_code=result1_latest["segments"], device=device)
result2_latest = whisperx.align(combined_data, model_a, metadata, audio_latest, device, return_char_alignments=False)

print( "Result 2 Latest: ", result2_latest["segments"]) # after alignment


# 3. Assign speaker labels
diarize_model_latest = whisperx.DiarizationPipeline(use_auth_token=write_api_key, device=device)

# add min/max number of speakers if known
diarize_segments_latest = diarize_model_latest(audio_latest)
diarize_model_latest(audio_latest, min_speakers=0, max_speakers=1)

result3_latest = whisperx.assign_word_speakers(diarize_segments_latest, result2_latest)
print( "Diarize Segments Latest: ", diarize_segments_latest)
print( "Result 3 Latest: ", result3_latest["segments"]) # segments are now assigned speaker IDs

-------------------------------------------------------------------------------------
{'start': 0.46000000834465027, 'end': 8.479999542236328, 'text': " Hello Farhan how are you? I'm fine how about you? I'm good let's talk about"}
{'start': 8.479999542236328, 'end': 14.680000305175781, 'text': " Fiverr success I'm recently created account on Fiverr but I have not"}
{'start': 14.680000305175781, 'end': 20.559999465942383, 'text': " success and can't get any order from there so can you please tell me some"}
{'start': 20.559999465942383, 'end': 26.940000534057617, 'text': ' guidance or some tips from how then I can succeed from there. Okay before'}
{'start': 26.940000534057617, 'end': 29.040000915527344, 'text': ' proceeding further I want to ask you what niche are you working in?'}
{'start': 29.040000915527344, 'end': 33.540000915527344, 'text': ' Currently I am providing my services on in data sciences and artificial intelligence. Wow'}
{'start': 33.540000915527344, 'end': 40.3800010681

In [58]:
transcript_latest

Transcription(text="Hello Farhan how are you? I'm fine how about you? I'm good let's talk about Fiverr success I'm recently created account on Fiverr but I have not success and can't get any order from there so can you please tell me some guidance or some tips from how then I can succeed from there. Okay before proceeding further I want to ask you what niche are you working in? Currently I am providing my services on in data sciences and artificial intelligence. Wow that's good okay so you are not getting impressions or orders? Both in impression I also can't get a more impression and the success rate is also not good. Okay so I would advise you that you should go for sharing your Fiverr profile in your whatsapp group Facebook groups that is the first step in the in our next call I will tell you about the second tip that that is all for today. Is it feasible for me or I can attempt some more techniques like? No for now just focus on this technique okay and then I will tell you the more

In [62]:
import json
import math

# Sample transcription detail
# transcription_detail = {
#   "task": "transcribe",
#   "language": "english",
#   "duration": 8.470000267028809,
#   "text": "The beach was a popular spot on a hot summer day. People were swimming in the ocean, building sandcastles, and playing beach volleyball.",
#   "segments": [
#     {
#       "id": 0,
#       "seek": 0,
#       "start": 0.0,
#       "end": 3.319999933242798,
#       "text": " The beach was a popular spot on a hot summer day.",
#       "tokens": [
#         50364, 440, 7534, 390, 257, 3743, 4008, 322, 257, 2368, 4266, 786, 13, 50530
#       ],
#       "temperature": 0.0,
#       "avg_logprob": -0.2860786020755768,
#       "compression_ratio": 1.2363636493682861,
#       "no_speech_prob": 0.00985979475080967
#     }
#   ]
# }

transcription_detail = {
    "task":'transcribe', 
    "language":'english', 
    "duration":79.8499984741211,
    "text":"Hello Farhan how are you? I'm fine how about you? I'm good let's talk about Fiverr success I'm recently created account on Fiverr but I have not success and can't get any order from there so can you please tell me some guidance or some tips from how then I can succeed from there. Okay before proceeding further I want to ask you what niche are you working in? Currently I am providing my services on in data sciences and artificial intelligence. Wow that's good okay so you are not getting impressions or orders? Both in impression I also can't get a more impression and the success rate is also not good. Okay so I would advise you that you should go for sharing your Fiverr profile in your whatsapp group Facebook groups that is the first step in the in our next call I will tell you about the second tip that that is all for today. Is it feasible for me or I can attempt some more techniques like? No for now just focus on this technique okay and then I will tell you the more secrets. Okay thanks for guiding me and thank you Amad thank you",
    "segments":[
      {
        'id': 0, 
        'seek': 0, 
        'start': 0.46000000834465027, 
        'end': 8.479999542236328, 
        'text': " Hello Farhan how are you? I'm fine how about you? I'm good let's talk about", 
        'tokens': [
            50364, 2425, 9067, 3451, 577, 366, 291, 30, 286, 478, 2489, 577, 466, 291, 30, 286, 478, 665, 718, 311, 751, 466, 50724
            ], 
            'temperature': 0.0, 
            'avg_logprob': -0.3786287009716034, 
            'compression_ratio': 1.6513761281967163, 
            'no_speech_prob': 0.9301366209983826
            }
    ]
}


# Function to calculate segment-level confidence
def calculate_segment_confidence(transcription_detail):
    # Extract avg_logprob values from all segments
    avg_logprobs = [segment["avg_logprob"] for segment in transcription_detail["segments"]]
    
    # Calculate the sum of avg_logprob values
    sum_avg_logprobs = sum(avg_logprobs)
    
    # Calculate the exponential of the sum of avg_logprob values
    segment_confidence = math.exp(sum_avg_logprobs)
    
    return segment_confidence

# Calculate segment-level confidence
confidence = calculate_segment_confidence(transcription_detail)
print("Segment-Level Confidence (Exponential of Sum of Avg_Logprobs):", confidence)


Segment-Level Confidence (Exponential of Sum of Avg_Logprobs): 0.6847998309797402


In [53]:
def calculate_word_scores(words):
    letter_scores = {chr(i+96): i for i in range(1, 27)}
    word_scores = []
    for word in words:
        score = sum(letter_scores[letter] for letter in word)
        word_scores.append(score)
    return word_scores

words = ["dog", "cat", "python", "fiverr"]
print(calculate_word_scores(words))

[26, 24, 98, 78]


In [7]:
def extract_data(json_data):
    for segment in json_data:
       output = print(segment['speaker'] + ": " + segment['text'])
    return output

extract_data(result3_latest['segments'])

SPEAKER_00:  السلام علیکم میں حبیبینک کا نمائندہ ہوں ہم آپ کو کارڈ لینے کی تجویز کریں گے کیا آپ انڈسٹر ہیں؟
SPEAKER_01:  وعلیکم سلام جی ضرور پلیس بتائیے آپ کے پاس کون سے کریڈ کارڈز اویلیبل ہیں؟
SPEAKER_00:  ہمارے کریڈ کارڈ میں کافی آسانیاں ہیں رقم نکروانے سے لے کر اور بھی بہت سی ایسی خاصوصیت ہیں جو دوسرے بینک کے کارڈ سے ہمیں الگ بناتی ہیں
SPEAKER_03:  اچھا یہ تو بہت اچھی بات ہے پلیس کیا آپ ڈیٹیئر شیئر کر سکتے ہیں تاکہ میں اس کو دیکھ سکوں؟
SPEAKER_00:  ٹھیک ہے آپ ہمارے ویب سائٹ پر فارم فل کر دیں اس کے بعد ساری ڈیٹیئر آپ کو مل جائیں گے بہت شکریہ
SPEAKER_02:  تو کیا میں آپ کو پرسنلی میسج کر سکتا ہوں؟
SPEAKER_00:  نہیں بالکل نہیں آپ ہمارے ویب سائٹ پر جا کے فارم فل کریں
SPEAKER_02:  ٹھیک ہے میں ویب سائٹ پر جا کے کرتا ہوں تینکیو


In [28]:
wrong_data = [{'text': ' Hi Sir, How are you?', 'start': 0.0, 'end': 3.0}, {'text': ' Hello, May I know who is speaking?', 'start': 3.0, 'end': 7.0}, {'text': ' I am speaking from the youth bank.', 'start': 7.0, 'end': 10.0}, {'text': ' Just wanted to tell you that recently we have started a credit card scheme.', 'start': 10.0, 'end': 15.0}, {'text': ' Let me know if you need one.', 'start': 15.0, 'end': 17.0}, {'text': ' Yes, I am interested in debit card.', 'start': 17.0, 'end': 20.0}, {'text': ' Please share the details in Urdu.', 'start': 20.0, 'end': 23.0}, {'text': ' Our debit cards are cheaper than others and we also give 30 days money back guarantee.', 'start': 23.0, 'end': 30.0}, {'text': ' Okay, so what is the process of taking the debit card?', 'start': 30.0, 'end': 35.0}, {'text': ' You have two options.', 'start': 35.0, 'end': 37.0}, {'text': " Either you can officially fill the form on the bank's website and apply for the debit card.", 'start': 37.0, 'end': 43.0}, {'text': ' But it has more charges.', 'start': 43.0, 'end': 45.0}, {'text': ' And if you take it directly from me, then you will have to pay less.', 'start': 45.0, 'end': 50.0}, {'text': ' Okay, so what do I have to do to take it from you?', 'start': 50.0, 'end': 55.0}, {'text': ' Please send me your details on the number.', 'start': 55.0, 'end': 59.0}, {'text': ' Okay, I will send you the details on WhatsApp.', 'start': 59.0, 'end': 63.0}, {'text': ' Thank you.', 'start': 63.0, 'end': 65.0}]

for i in range(len(wrong_data)):
    print(wrong_data[i])

{'text': ' Hi Sir, How are you?', 'start': 0.0, 'end': 3.0}
{'text': ' Hello, May I know who is speaking?', 'start': 3.0, 'end': 7.0}
{'text': ' I am speaking from the youth bank.', 'start': 7.0, 'end': 10.0}
{'text': ' Just wanted to tell you that recently we have started a credit card scheme.', 'start': 10.0, 'end': 15.0}
{'text': ' Let me know if you need one.', 'start': 15.0, 'end': 17.0}
{'text': ' Yes, I am interested in debit card.', 'start': 17.0, 'end': 20.0}
{'text': ' Please share the details in Urdu.', 'start': 20.0, 'end': 23.0}
{'text': ' Our debit cards are cheaper than others and we also give 30 days money back guarantee.', 'start': 23.0, 'end': 30.0}
{'text': ' Okay, so what is the process of taking the debit card?', 'start': 30.0, 'end': 35.0}
{'text': ' You have two options.', 'start': 35.0, 'end': 37.0}
{'text': " Either you can officially fill the form on the bank's website and apply for the debit card.", 'start': 37.0, 'end': 43.0}
{'text': ' But it has more charg

In [27]:
# Rearrange the sequence of each dictionary element
corrected_data = [{'start': item['start'], 'end': item['end'], 'text': item['text']} for item in wrong_data]

# Printing the corrected data
for item in corrected_data:
    print(item)

{'start': 0.0, 'end': 3.0, 'text': ' Hi Sir, How are you?'}
{'start': 3.0, 'end': 7.0, 'text': ' Hello, May I know who is speaking?'}
{'start': 7.0, 'end': 10.0, 'text': ' I am speaking from the youth bank.'}
{'start': 10.0, 'end': 15.0, 'text': ' Just wanted to tell you that recently we have started a credit card scheme.'}
{'start': 15.0, 'end': 17.0, 'text': ' Let me know if you need one.'}
{'start': 17.0, 'end': 20.0, 'text': ' Yes, I am interested in debit card.'}
{'start': 20.0, 'end': 23.0, 'text': ' Please share the details in Urdu.'}
{'start': 23.0, 'end': 30.0, 'text': ' Our debit cards are cheaper than others and we also give 30 days money back guarantee.'}
{'start': 30.0, 'end': 35.0, 'text': ' Okay, so what is the process of taking the debit card?'}
{'start': 35.0, 'end': 37.0, 'text': ' You have two options.'}
{'start': 37.0, 'end': 43.0, 'text': " Either you can officially fill the form on the bank's website and apply for the debit card."}
{'start': 43.0, 'end': 45.0, 'te

## Now adding words list into above:

In [29]:
words = [{'word': 'Hello', 'start': 0.46000000834465027, 'end': 1.1399999856948853}, {'word': 'Farhan', 'start': 1.1399999856948853, 'end': 1.7400000095367432}, {'word': 'how', 'start': 1.7400000095367432, 'end': 1.8600000143051147}, {'word': 'are', 'start': 1.8600000143051147, 'end': 2.0199999809265137}, {'word': 'you', 'start': 2.0199999809265137, 'end': 2.359999895095825}, {'word': "I'm", 'start': 2.759999990463257, 'end': 2.940000057220459}, {'word': 'fine', 'start': 2.940000057220459, 'end': 3.140000104904175}, {'word': 'how', 'start': 3.140000104904175, 'end': 3.4000000953674316}, {'word': 'about', 'start': 3.4000000953674316, 'end': 4.139999866485596}, {'word': 'you', 'start': 4.139999866485596, 'end': 4.800000190734863}, {'word': "I'm", 'start': 5.28000020980835, 'end': 5.380000114440918}, {'word': 'good', 'start': 5.380000114440918, 'end': 5.599999904632568}, {'word': "let's", 'start': 5.599999904632568, 'end': 6.739999771118164}, {'word': 'talk', 'start': 6.739999771118164, 'end': 6.940000057220459}, {'word': 'about', 'start': 6.940000057220459, 'end': 8.020000457763672}, {'word': 'Fiverr', 'start': 8.020000457763672, 'end': 8.479999542236328}, {'word': 'success', 'start': 8.479999542236328, 'end': 8.84000015258789}, {'word': "I'm", 'start': 8.84000015258789, 'end': 9.34000015258789}, {'word': 'recently', 'start': 9.34000015258789, 'end': 9.720000267028809}, {'word': 'created', 'start': 9.720000267028809, 'end': 10.380000114440918}, {'word': 'account', 'start': 10.380000114440918, 'end': 10.680000305175781}, {'word': 'on', 'start': 10.680000305175781, 'end': 10.880000114440918}, {'word': 'Fiverr', 'start': 10.880000114440918, 'end': 11.380000114440918}, {'word': 'but', 'start': 11.380000114440918, 'end': 11.739999771118164}, {'word': 'I', 'start': 11.739999771118164, 'end': 12.079999923706055}, {'word': 'have', 'start': 12.079999923706055, 'end': 12.220000267028809}, {'word': 'not', 'start': 12.220000267028809, 'end': 12.4399995803833}, {'word': 'success', 'start': 12.4399995803833, 'end': 12.9399995803833}, {'word': 'and', 'start': 12.9399995803833, 'end': 14.420000076293945}, {'word': "can't", 'start': 14.420000076293945, 'end': 14.680000305175781}, {'word': 'get', 'start': 14.680000305175781, 'end': 14.859999656677246}, {'word': 'any', 'start': 14.859999656677246, 'end': 15.0600004196167}, {'word': 'order', 'start': 15.0600004196167, 'end': 15.239999771118164}, {'word': 'from', 'start': 15.239999771118164, 'end': 15.420000076293945}, {'word': 'there', 'start': 15.420000076293945, 'end': 15.680000305175781}, {'word': 'so', 'start': 15.680000305175781, 'end': 16.139999389648438}, {'word': 'can', 'start': 16.139999389648438, 'end': 16.260000228881836}, {'word': 'you', 'start': 16.260000228881836, 'end': 16.399999618530273}, {'word': 'please', 'start': 16.399999618530273, 'end': 16.65999984741211}, {'word': 'tell', 'start': 16.65999984741211, 'end': 17.059999465942383}, {'word': 'me', 'start': 17.059999465942383, 'end': 17.219999313354492}, {'word': 'some', 'start': 17.219999313354492, 'end': 18.239999771118164}, {'word': 'guidance', 'start': 18.239999771118164, 'end': 18.739999771118164}, {'word': 'or', 'start': 18.739999771118164, 'end': 18.920000076293945}, {'word': 'some', 'start': 18.920000076293945, 'end': 19.219999313354492}, {'word': 'tips', 'start': 19.219999313354492, 'end': 19.360000610351562}, {'word': 'from', 'start': 19.360000610351562, 'end': 19.700000762939453}, {'word': 'how', 'start': 19.700000762939453, 'end': 20.559999465942383}, {'word': 'then', 'start': 20.559999465942383, 'end': 20.979999542236328}, {'word': 'I', 'start': 20.979999542236328, 'end': 21.31999969482422}, {'word': 'can', 'start': 21.31999969482422, 'end': 21.559999465942383}, {'word': 'succeed', 'start': 21.559999465942383, 'end': 21.940000534057617}, {'word': 'from', 'start': 21.940000534057617, 'end': 22.18000030517578}, {'word': 'there', 'start': 22.18000030517578, 'end': 22.479999542236328}, {'word': 'Okay', 'start': 22.739999771118164, 'end': 23.18000030517578}, {'word': 'before', 'start': 23.18000030517578, 'end': 23.5}, {'word': 'proceeding', 'start': 23.5, 'end': 23.920000076293945}, {'word': 'further', 'start': 23.920000076293945, 'end': 24.280000686645508}, {'word': 'I', 'start': 24.280000686645508, 'end': 24.6200008392334}, {'word': 'want', 'start': 24.6200008392334, 'end': 24.860000610351562}, {'word': 'to', 'start': 24.860000610351562, 'end': 25.200000762939453}, {'word': 'ask', 'start': 25.200000762939453, 'end': 25.459999084472656}, {'word': 'you', 'start': 25.459999084472656, 'end': 26.100000381469727}, {'word': 'what', 'start': 26.100000381469727, 'end': 26.940000534057617}, {'word': 'niche', 'start': 26.940000534057617, 'end': 27.8799991607666}, {'word': 'are', 'start': 27.8799991607666, 'end': 28.1200008392334}, {'word': 'you', 'start': 28.1200008392334, 'end': 28.459999084472656}, {'word': 'working', 'start': 28.459999084472656, 'end': 28.920000076293945}, {'word': 'in', 'start': 28.920000076293945, 'end': 29.040000915527344}, {'word': 'Currently', 'start': 29.040000915527344, 'end': 29.280000686645508}, {'word': 'I', 'start': 29.280000686645508, 'end': 29.6200008392334}, {'word': 'am', 'start': 29.6200008392334, 'end': 30.0}, {'word': 'providing', 'start': 30.0, 'end': 30.0}, {'word': 'my', 'start': 30.0, 'end': 30.200000762939453}, {'word': 'services', 'start': 30.200000762939453, 'end': 30.459999084472656}, {'word': 'on', 'start': 30.459999084472656, 'end': 30.84000015258789}, {'word': 'in', 'start': 30.84000015258789, 'end': 31.020000457763672}, {'word': 'data', 'start': 31.020000457763672, 'end': 31.15999984741211}, {'word': 'sciences', 'start': 31.15999984741211, 'end': 31.479999542236328}, {'word': 'and', 'start': 31.479999542236328, 'end': 31.700000762939453}, {'word': 'artificial', 'start': 31.700000762939453, 'end': 32.08000183105469}, {'word': 'intelligence', 'start': 32.08000183105469, 'end': 32.779998779296875}, {'word': 'Wow', 'start': 32.84000015258789, 'end': 33.099998474121094}, {'word': "that's", 'start': 33.099998474121094, 'end': 33.540000915527344}, {'word': 'good', 'start': 33.540000915527344, 'end': 34.36000061035156}, {'word': 'okay', 'start': 34.36000061035156, 'end': 34.939998626708984}, {'word': 'so', 'start': 34.939998626708984, 'end': 35.279998779296875}, {'word': 'you', 'start': 35.279998779296875, 'end': 35.459999084472656}, {'word': 'are', 'start': 35.459999084472656, 'end': 35.619998931884766}, {'word': 'not', 'start': 35.619998931884766, 'end': 35.7599983215332}, {'word': 'getting', 'start': 35.7599983215332, 'end': 35.97999954223633}, {'word': 'impressions', 'start': 35.97999954223633, 'end': 36.439998626708984}, {'word': 'or', 'start': 36.439998626708984, 'end': 36.81999969482422}, {'word': 'orders', 'start': 36.81999969482422, 'end': 37.380001068115234}, {'word': 'Both', 'start': 38.08000183105469, 'end': 38.41999816894531}, {'word': 'in', 'start': 38.41999816894531, 'end': 39.060001373291016}, {'word': 'impression', 'start': 39.060001373291016, 'end': 39.380001068115234}, {'word': 'I', 'start': 39.380001068115234, 'end': 39.619998931884766}, {'word': 'also', 'start': 39.619998931884766, 'end': 40.040000915527344}, {'word': "can't", 'start': 40.040000915527344, 'end': 40.380001068115234}, {'word': 'get', 'start': 40.380001068115234, 'end': 40.63999938964844}, {'word': 'a', 'start': 40.63999938964844, 'end': 40.81999969482422}, {'word': 'more', 'start': 40.81999969482422, 'end': 41.400001525878906}, {'word': 'impression', 'start': 41.400001525878906, 'end': 41.720001220703125}, {'word': 'and', 'start': 41.720001220703125, 'end': 42.15999984741211}, {'word': 'the', 'start': 42.15999984741211, 'end': 42.52000045776367}, {'word': 'success', 'start': 42.52000045776367, 'end': 42.86000061035156}, {'word': 'rate', 'start': 42.86000061035156, 'end': 43.119998931884766}, {'word': 'is', 'start': 43.119998931884766, 'end': 43.2599983215332}, {'word': 'also', 'start': 43.2599983215332, 'end': 43.5}, {'word': 'not', 'start': 43.5, 'end': 43.7599983215332}, {'word': 'good', 'start': 43.7599983215332, 'end': 44.31999969482422}, {'word': 'Okay', 'start': 44.540000915527344, 'end': 45.02000045776367}, {'word': 'so', 'start': 45.02000045776367, 'end': 45.540000915527344}, {'word': 'I', 'start': 45.540000915527344, 'end': 46.040000915527344}, {'word': 'would', 'start': 46.040000915527344, 'end': 46.2599983215332}, {'word': 'advise', 'start': 46.2599983215332, 'end': 46.959999084472656}, {'word': 'you', 'start': 46.959999084472656, 'end': 47.18000030517578}, {'word': 'that', 'start': 47.18000030517578, 'end': 47.459999084472656}, {'word': 'you', 'start': 47.459999084472656, 'end': 48.18000030517578}, {'word': 'should', 'start': 48.18000030517578, 'end': 48.439998626708984}, {'word': 'go', 'start': 48.439998626708984, 'end': 49.68000030517578}, {'word': 'for', 'start': 49.68000030517578, 'end': 50.119998931884766}, {'word': 'sharing', 'start': 50.119998931884766, 'end': 51.040000915527344}, {'word': 'your', 'start': 51.040000915527344, 'end': 51.400001525878906}, {'word': 'Fiverr', 'start': 51.400001525878906, 'end': 51.63999938964844}, {'word': 'profile', 'start': 51.63999938964844, 'end': 51.959999084472656}, {'word': 'in', 'start': 51.959999084472656, 'end': 52.220001220703125}, {'word': 'your', 'start': 52.220001220703125, 'end': 52.439998626708984}, {'word': 'whatsapp', 'start': 52.439998626708984, 'end': 52.720001220703125}, {'word': 'group', 'start': 52.720001220703125, 'end': 52.959999084472656}, {'word': 'Facebook', 'start': 52.959999084472656, 'end': 53.2599983215332}, {'word': 'groups', 'start': 53.2599983215332, 'end': 53.560001373291016}, {'word': 'that', 'start': 53.560001373291016, 'end': 54.31999969482422}, {'word': 'is', 'start': 54.31999969482422, 'end': 54.47999954223633}, {'word': 'the', 'start': 54.47999954223633, 'end': 54.63999938964844}, {'word': 'first', 'start': 54.63999938964844, 'end': 54.84000015258789}, {'word': 'step', 'start': 54.84000015258789, 'end': 55.119998931884766}, {'word': 'in', 'start': 55.119998931884766, 'end': 55.63999938964844}, {'word': 'the', 'start': 55.63999938964844, 'end': 55.81999969482422}, {'word': 'in', 'start': 55.81999969482422, 'end': 56.099998474121094}, {'word': 'our', 'start': 56.099998474121094, 'end': 56.2400016784668}, {'word': 'next', 'start': 56.2400016784668, 'end': 56.560001373291016}, {'word': 'call', 'start': 56.939998626708984, 'end': 57.58000183105469}, {'word': 'I', 'start': 57.58000183105469, 'end': 57.86000061035156}, {'word': 'will', 'start': 57.86000061035156, 'end': 58.29999923706055}, {'word': 'tell', 'start': 58.29999923706055, 'end': 58.91999816894531}, {'word': 'you', 'start': 58.91999816894531, 'end': 59.040000915527344}, {'word': 'about', 'start': 59.040000915527344, 'end': 59.279998779296875}, {'word': 'the', 'start': 59.279998779296875, 'end': 59.540000915527344}, {'word': 'second', 'start': 59.540000915527344, 'end': 59.900001525878906}, {'word': 'tip', 'start': 59.900001525878906, 'end': 60.70000076293945}, {'word': 'that', 'start': 60.70000076293945, 'end': 61.279998779296875}, {'word': 'that', 'start': 61.279998779296875, 'end': 61.560001373291016}, {'word': 'is', 'start': 61.560001373291016, 'end': 61.599998474121094}, {'word': 'all', 'start': 61.599998474121094, 'end': 61.84000015258789}, {'word': 'for', 'start': 61.84000015258789, 'end': 62.040000915527344}, {'word': 'today', 'start': 62.040000915527344, 'end': 62.36000061035156}, {'word': 'Is', 'start': 62.36000061035156, 'end': 62.560001373291016}, {'word': 'it', 'start': 62.560001373291016, 'end': 62.720001220703125}, {'word': 'feasible', 'start': 62.720001220703125, 'end': 63.18000030517578}, {'word': 'for', 'start': 63.18000030517578, 'end': 63.400001525878906}, {'word': 'me', 'start': 63.400001525878906, 'end': 63.7599983215332}, {'word': 'or', 'start': 63.7599983215332, 'end': 64.4000015258789}, {'word': 'I', 'start': 64.4000015258789, 'end': 64.87999725341797}, {'word': 'can', 'start': 64.87999725341797, 'end': 65.86000061035156}, {'word': 'attempt', 'start': 65.86000061035156, 'end': 66.36000061035156}, {'word': 'some', 'start': 66.36000061035156, 'end': 66.63999938964844}, {'word': 'more', 'start': 66.63999938964844, 'end': 67.18000030517578}, {'word': 'techniques', 'start': 67.18000030517578, 'end': 67.5199966430664}, {'word': 'like', 'start': 67.5199966430664, 'end': 67.87999725341797}, {'word': 'No', 'start': 68.23999786376953, 'end': 68.5199966430664}, {'word': 'for', 'start': 68.5199966430664, 'end': 68.9800033569336}, {'word': 'now', 'start': 68.9800033569336, 'end': 69.31999969482422}, {'word': 'just', 'start': 69.31999969482422, 'end': 69.72000122070312}, {'word': 'focus', 'start': 69.72000122070312, 'end': 70.0}, {'word': 'on', 'start': 70.0, 'end': 70.19999694824219}, {'word': 'this', 'start': 70.19999694824219, 'end': 70.45999908447266}, {'word': 'technique', 'start': 70.45999908447266, 'end': 70.72000122070312}, {'word': 'okay', 'start': 70.72000122070312, 'end': 71.26000213623047}, {'word': 'and', 'start': 71.26000213623047, 'end': 71.58000183105469}, {'word': 'then', 'start': 71.58000183105469, 'end': 71.77999877929688}, {'word': 'I', 'start': 71.77999877929688, 'end': 71.94000244140625}, {'word': 'will', 'start': 71.94000244140625, 'end': 72.0999984741211}, {'word': 'tell', 'start': 72.0999984741211, 'end': 72.26000213623047}, {'word': 'you', 'start': 72.26000213623047, 'end': 72.37999725341797}, {'word': 'the', 'start': 72.37999725341797, 'end': 72.58000183105469}, {'word': 'more', 'start': 72.58000183105469, 'end': 72.81999969482422}, {'word': 'secrets', 'start': 72.81999969482422, 'end': 73.18000030517578}, {'word': 'Okay', 'start': 73.44000244140625, 'end': 73.72000122070312}, {'word': 'thanks', 'start': 73.72000122070312, 'end': 74.12000274658203}, {'word': 'for', 'start': 74.12000274658203, 'end': 74.69999694824219}, {'word': 'guiding', 'start': 74.69999694824219, 'end': 75.68000030517578}, {'word': 'me', 'start': 75.68000030517578, 'end': 76.0999984741211}, {'word': 'and', 'start': 76.0999984741211, 'end': 76.72000122070312}, {'word': 'thank', 'start': 76.72000122070312, 'end': 77.95999908447266}, {'word': 'you', 'start': 77.95999908447266, 'end': 78.08000183105469}, {'word': 'Amad', 'start': 78.08000183105469, 'end': 78.26000213623047}, {'word': 'thank', 'start': 78.26000213623047, 'end': 78.58000183105469}, {'word': 'you', 'start': 78.58000183105469, 'end': 78.86000061035156}]

for i in range(len(words)):
    print(words[i])

{'word': 'Hello', 'start': 0.46000000834465027, 'end': 1.1399999856948853}
{'word': 'Farhan', 'start': 1.1399999856948853, 'end': 1.7400000095367432}
{'word': 'how', 'start': 1.7400000095367432, 'end': 1.8600000143051147}
{'word': 'are', 'start': 1.8600000143051147, 'end': 2.0199999809265137}
{'word': 'you', 'start': 2.0199999809265137, 'end': 2.359999895095825}
{'word': "I'm", 'start': 2.759999990463257, 'end': 2.940000057220459}
{'word': 'fine', 'start': 2.940000057220459, 'end': 3.140000104904175}
{'word': 'how', 'start': 3.140000104904175, 'end': 3.4000000953674316}
{'word': 'about', 'start': 3.4000000953674316, 'end': 4.139999866485596}
{'word': 'you', 'start': 4.139999866485596, 'end': 4.800000190734863}
{'word': "I'm", 'start': 5.28000020980835, 'end': 5.380000114440918}
{'word': 'good', 'start': 5.380000114440918, 'end': 5.599999904632568}
{'word': "let's", 'start': 5.599999904632568, 'end': 6.739999771118164}
{'word': 'talk', 'start': 6.739999771118164, 'end': 6.94000005722045

In [30]:
# Combine data from wrong_data and words
combined_data = []
for idx, data in enumerate(wrong_data):
    data_copy = data.copy()
    data_copy['words'] = []
    for word in words:
        if data['start'] <= word['start'] <= data['end']:
            data_copy['words'].append(word)
    combined_data.append(data_copy)

# Print the combined data
for item in combined_data:
    print(item)

{'text': ' Hi Sir, How are you?', 'start': 0.0, 'end': 3.0, 'words': [{'word': 'Hello', 'start': 0.46000000834465027, 'end': 1.1399999856948853}, {'word': 'Farhan', 'start': 1.1399999856948853, 'end': 1.7400000095367432}, {'word': 'how', 'start': 1.7400000095367432, 'end': 1.8600000143051147}, {'word': 'are', 'start': 1.8600000143051147, 'end': 2.0199999809265137}, {'word': 'you', 'start': 2.0199999809265137, 'end': 2.359999895095825}, {'word': "I'm", 'start': 2.759999990463257, 'end': 2.940000057220459}, {'word': 'fine', 'start': 2.940000057220459, 'end': 3.140000104904175}]}
{'text': ' Hello, May I know who is speaking?', 'start': 3.0, 'end': 7.0, 'words': [{'word': 'how', 'start': 3.140000104904175, 'end': 3.4000000953674316}, {'word': 'about', 'start': 3.4000000953674316, 'end': 4.139999866485596}, {'word': 'you', 'start': 4.139999866485596, 'end': 4.800000190734863}, {'word': "I'm", 'start': 5.28000020980835, 'end': 5.380000114440918}, {'word': 'good', 'start': 5.380000114440918, 

In [31]:
combined_data

[{'text': ' Hi Sir, How are you?',
  'start': 0.0,
  'end': 3.0,
  'words': [{'word': 'Hello',
    'start': 0.46000000834465027,
    'end': 1.1399999856948853},
   {'word': 'Farhan', 'start': 1.1399999856948853, 'end': 1.7400000095367432},
   {'word': 'how', 'start': 1.7400000095367432, 'end': 1.8600000143051147},
   {'word': 'are', 'start': 1.8600000143051147, 'end': 2.0199999809265137},
   {'word': 'you', 'start': 2.0199999809265137, 'end': 2.359999895095825},
   {'word': "I'm", 'start': 2.759999990463257, 'end': 2.940000057220459},
   {'word': 'fine', 'start': 2.940000057220459, 'end': 3.140000104904175}]},
 {'text': ' Hello, May I know who is speaking?',
  'start': 3.0,
  'end': 7.0,
  'words': [{'word': 'how',
    'start': 3.140000104904175,
    'end': 3.4000000953674316},
   {'word': 'about', 'start': 3.4000000953674316, 'end': 4.139999866485596},
   {'word': 'you', 'start': 4.139999866485596, 'end': 4.800000190734863},
   {'word': "I'm", 'start': 5.28000020980835, 'end': 5.38000

In [20]:
want_data = [{'start': 0.0, 'end': 7.0, 'text': ' السلام علیکم میں حبیبینک کا نمائندہ ہوں ہم آپ کو کارڈ لینے کی تجویز کریں گے کیا آپ انڈسٹر ہیں؟', 'words': [{'word': 'السلام', 'start': 0.0, 'end': 1.324, 'score': 0.164}, {'word': 'علیکم', 'start': 1.464, 'end': 1.825, 'score': 0.262}, {'word': 'میں', 'start': 1.926, 'end': 2.046, 'score': 0.403}, {'word': 'حبیبینک', 'start': 2.106, 'end': 2.668, 'score': 0.335}, {'word': 'کا', 'start': 2.688, 'end': 2.768, 'score': 0.603}, {'word': 'نمائندہ', 'start': 2.808, 'end': 3.269, 'score': 0.233}, {'word': 'ہوں', 'start': 3.289, 'end': 3.47, 'score': 0.083}, {'word': 'ہم', 'start': 3.59, 'end': 3.731, 'score': 0.567}, {'word': 'آپ', 'start': 3.751, 'end': 3.851, 'score': 0.006}, {'word': 'کو', 'start': 3.891, 'end': 3.971, 'score': 0.254}, {'word': 'کارڈ', 'start': 4.032, 'end': 4.252, 'score': 0.064}, {'word': 'لینے', 'start': 4.272, 'end': 4.473, 'score': 0.203}, {'word': 'کی', 'start': 4.493, 'end': 4.573, 'score': 0.279}, {'word': 'تجویز', 'start': 4.613, 'end': 5.074, 'score': 0.16}, {'word': 'کریں', 'start': 5.095, 'end': 5.275, 'score': 0.453}, {'word': 'گے', 'start': 5.315, 'end': 5.436, 'score': 0.164}, {'word': 'کیا', 'start': 5.857, 'end': 6.037, 'score': 0.363}, {'word': 'آپ', 'start': 6.117, 'end': 6.178, 'score': 0.022}, {'word': 'انڈسٹر', 'start': 6.198, 'end': 6.639, 'score': 0.198}, {'word': 'ہیں؟', 'start': 6.94, 'end': 7.0, 'score': 0.288}]}, {'start': 7.0, 'end': 12.0, 'text': ' وعلیکم سلام جی ضرور پلیس بتائیے آپ کے پاس کون سے کریڈ کارڈز اویلیبل ہیں؟', 'words': [{'word': 'وعلیکم', 'start': 7.0, 'end': 8.004, 'score': 0.202}, {'word': 'سلام', 'start': 8.024, 'end': 8.365, 'score': 0.179}, {'word': 'جی', 'start': 8.466, 'end': 8.667, 'score': 0.286}, {'word': 'ضرور', 'start': 8.707, 'end': 9.028, 'score': 0.295}, {'word': 'پلیس', 'start': 9.49, 'end': 9.711, 'score': 0.164}, {'word': 'بتائیے', 'start': 9.751, 'end': 10.112, 'score': 0.321}, {'word': 'آپ', 'start': 10.133, 'end': 10.213, 'score': 0.001}, {'word': 'کے', 'start': 10.253, 'end': 10.313, 'score': 0.742}, {'word': 'پاس', 'start': 10.373, 'end': 10.614, 'score': 0.265}, {'word': 'کون', 'start': 10.655, 'end': 10.855, 'score': 0.303}, {'word': 'سے', 'start': 10.916, 'end': 11.016, 'score': 0.468}, {'word': 'کریڈ', 'start': 11.076, 'end': 11.277, 'score': 0.415}, {'word': 'کارڈز', 'start': 11.337, 'end': 11.578, 'score': 0.472}, {'word': 'اویلیبل', 'start': 11.639, 'end': 11.92, 'score': 0.224}, {'word': 'ہیں؟', 'start': 11.94, 'end': 12.0, 'score': 0.029}]}, {'start': 12.0, 'end': 20.0, 'text': ' ہمارے کریڈ کارڈ میں کافی آسانیاں ہیں رقم نکروانے سے لے کر اور بھی بہت سی ایسی خاصوصیت ہیں جو دوسرے بینک کے کارڈ سے ہمیں الگ بناتی ہیں', 'words': [{'word': 'ہمارے', 'start': 12.0, 'end': 13.063, 'score': 0.314}, {'word': 'کریڈ', 'start': 13.143, 'end': 13.363, 'score': 0.226}, {'word': 'کارڈ', 'start': 13.404, 'end': 13.584, 'score': 0.294}, {'word': 'میں', 'start': 13.604, 'end': 13.664, 'score': 0.258}, {'word': 'کافی', 'start': 13.704, 'end': 13.985, 'score': 0.247}, {'word': 'آسانیاں', 'start': 14.005, 'end': 14.426, 'score': 0.11}, {'word': 'ہیں', 'start': 14.526, 'end': 14.707, 'score': 0.068}, {'word': 'رقم', 'start': 15.268, 'end': 15.489, 'score': 0.445}, {'word': 'نکروانے', 'start': 15.529, 'end': 15.93, 'score': 0.317}, {'word': 'سے', 'start': 15.97, 'end': 16.03, 'score': 0.745}, {'word': 'لے', 'start': 16.07, 'end': 16.211, 'score': 0.13}, {'word': 'کر', 'start': 16.231, 'end': 16.391, 'score': 0.256}, {'word': 'اور', 'start': 16.471, 'end': 16.652, 'score': 0.442}, {'word': 'بھی', 'start': 16.692, 'end': 16.772, 'score': 0.822}, {'word': 'بہت', 'start': 16.812, 'end': 17.053, 'score': 0.084}, {'word': 'سی', 'start': 17.073, 'end': 17.133, 'score': 0.126}, {'word': 'ایسی', 'start': 17.153, 'end': 17.333, 'score': 0.207}, {'word': 'خاصوصیت', 'start': 17.393, 'end': 17.835, 'score': 0.255}, {'word': 'ہیں', 'start': 17.915, 'end': 18.035, 'score': 0.111}, {'word': 'جو', 'start': 18.075, 'end': 18.155, 'score': 0.022}, {'word': 'دوسرے', 'start': 18.195, 'end': 18.436, 'score': 0.298}, {'word': 'بینک', 'start': 18.456, 'end': 18.657, 'score': 0.272}, {'word': 'کے', 'start': 18.697, 'end': 18.777, 'score': 0.246}, {'word': 'کارڈ', 'start': 18.817, 'end': 19.058, 'score': 0.358}, {'word': 'سے', 'start': 19.098, 'end': 19.158, 'score': 0.554}, {'word': 'ہمیں', 'start': 19.198, 'end': 19.378, 'score': 0.175}, {'word': 'الگ', 'start': 19.398, 'end': 19.539, 'score': 0.213}, {'word': 'بناتی', 'start': 19.579, 'end': 19.9, 'score': 0.147}, {'word': 'ہیں', 'start': 19.94, 'end': 20.0, 'score': 0.031}]}, {'start': 20.0, 'end': 25.0, 'text': ' اچھا یہ تو بہت اچھی بات ہے پلیس کیا آپ ڈیٹیئر شیئر کر سکتے ہیں تاکہ میں اس کو دیکھ سکوں؟', 'words': [{'word': 'اچھا', 'start': 20.0, 'end': 20.863, 'score': 0.237}, {'word': 'یہ', 'start': 20.924, 'end': 21.004, 'score': 0.473}, {'word': 'تو', 'start': 21.044, 'end': 21.124, 'score': 0.216}, {'word': 'بہت', 'start': 21.145, 'end': 21.365, 'score': 0.163}, {'word': 'اچھی', 'start': 21.386, 'end': 21.586, 'score': 0.36}, {'word': 'بات', 'start': 21.627, 'end': 21.948, 'score': 0.197}, {'word': 'ہے', 'start': 21.968, 'end': 22.008, 'score': 0.002}, {'word': 'پلیس', 'start': 22.289, 'end': 22.49, 'score': 0.327}, {'word': 'کیا', 'start': 22.53, 'end': 22.711, 'score': 0.425}, {'word': 'آپ', 'start': 22.731, 'end': 22.771, 'score': 0.001}, {'word': 'ڈیٹیئر', 'start': 23.032, 'end': 23.253, 'score': 0.333}, {'word': 'شیئر', 'start': 23.373, 'end': 23.534, 'score': 0.347}, {'word': 'کر', 'start': 23.574, 'end': 23.675, 'score': 0.539}, {'word': 'سکتے', 'start': 23.755, 'end': 23.976, 'score': 0.359}, {'word': 'ہیں', 'start': 23.996, 'end': 24.056, 'score': 0.289}, {'word': 'تاکہ', 'start': 24.096, 'end': 24.297, 'score': 0.526}, {'word': 'میں', 'start': 24.337, 'end': 24.458, 'score': 0.505}, {'word': 'اس', 'start': 24.478, 'end': 24.558, 'score': 0.191}, {'word': 'کو', 'start': 24.598, 'end': 24.679, 'score': 0.166}, {'word': 'دیکھ', 'start': 24.699, 'end': 24.779, 'score': 0.0}, {'word': 'سکوں؟', 'start': 24.799, 'end': 25.0, 'score': 0.001}]}, {'start': 25.0, 'end': 31.0, 'text': ' ٹھیک ہے آپ ہماری ویب سائیٹ پر فارم فل کر دیں اس کے بعد ساری ڈیٹیئر آپ کو مل جائیں گے بہت شکریہ', 'words': [{'word': 'ٹھیک', 'start': 25.0, 'end': 26.124, 'score': 0.341}, {'word': 'ہے', 'start': 26.164, 'end': 26.264, 'score': 0.363}, {'word': 'آپ', 'start': 26.344, 'end': 26.505, 'score': 0.216}, {'word': 'ہماری', 'start': 26.545, 'end': 26.766, 'score': 0.254}, {'word': 'ویب', 'start': 26.786, 'end': 26.866, 'score': 0.02}, {'word': 'سائیٹ', 'start': 26.886, 'end': 27.127, 'score': 0.083}, {'word': 'پر', 'start': 27.147, 'end': 27.288, 'score': 0.486}, {'word': 'فارم', 'start': 27.388, 'end': 27.548, 'score': 0.319}, {'word': 'فل', 'start': 27.589, 'end': 27.709, 'score': 0.373}, {'word': 'کر', 'start': 27.749, 'end': 27.87, 'score': 0.249}, {'word': 'دیں', 'start': 27.91, 'end': 28.07, 'score': 0.095}, {'word': 'اس', 'start': 28.09, 'end': 28.151, 'score': 0.294}, {'word': 'کے', 'start': 28.191, 'end': 28.251, 'score': 0.7}, {'word': 'بعد', 'start': 28.291, 'end': 28.411, 'score': 0.116}, {'word': 'ساری', 'start': 28.492, 'end': 28.753, 'score': 0.381}, {'word': 'ڈیٹیئر', 'start': 28.773, 'end': 29.074, 'score': 0.226}, {'word': 'آپ', 'start': 29.134, 'end': 29.314, 'score': 0.281}, {'word': 'کو', 'start': 29.355, 'end': 29.435, 'score': 0.67}, {'word': 'مل', 'start': 29.756, 'end': 29.896, 'score': 0.396}, {'word': 'جائیں', 'start': 29.916, 'end': 30.117, 'score': 0.459}, {'word': 'گے', 'start': 30.137, 'end': 30.318, 'score': 0.186}, {'word': 'بہت', 'start': 30.458, 'end': 30.579, 'score': 0.17}, {'word': 'شکریہ', 'start': 30.619, 'end': 31.0, 'score': 0.205}]}, {'start': 31.0, 'end': 34.0, 'text': ' تو کیا میں آپ کو پرسنلی میسج کر سکتا ہوں؟', 'words': [{'word': 'تو', 'start': 31.0, 'end': 31.342, 'score': 0.0}, {'word': 'کیا', 'start': 31.423, 'end': 31.644, 'score': 0.227}, {'word': 'میں', 'start': 32.148, 'end': 32.248, 'score': 0.397}, {'word': 'آپ', 'start': 32.268, 'end': 32.389, 'score': 0.22}, {'word': 'کو', 'start': 32.45, 'end': 32.611, 'score': 0.25}, {'word': 'پرسنلی', 'start': 32.711, 'end': 33.054, 'score': 0.357}, {'word': 'میسج', 'start': 33.074, 'end': 33.315, 'score': 0.302}, {'word': 'کر', 'start': 33.356, 'end': 33.456, 'score': 0.42}, {'word': 'سکتا', 'start': 33.497, 'end': 33.758, 'score': 0.444}, {'word': 'ہوں؟', 'start': 33.779, 'end': 34.0, 'score': 0.7}]}, {'start': 34.0, 'end': 38.0, 'text': ' نہیں بالکل نہیں آپ ہماری ویب سائیٹ پر جا کے فارم فل کریں', 'words': [{'word': 'نہیں', 'start': 34.0, 'end': 34.623, 'score': 0.248}, {'word': 'بالکل', 'start': 34.663, 'end': 34.925, 'score': 0.435}, {'word': 'نہیں', 'start': 34.945, 'end': 35.146, 'score': 0.076}, {'word': 'آپ', 'start': 35.648, 'end': 35.769, 'score': 0.004}, {'word': 'ہماری', 'start': 35.789, 'end': 36.03, 'score': 0.326}, {'word': 'ویب', 'start': 36.07, 'end': 36.271, 'score': 0.001}, {'word': 'سائیٹ', 'start': 36.653, 'end': 37.075, 'score': 0.204}, {'word': 'پر', 'start': 37.095, 'end': 37.136, 'score': 0.14}, {'word': 'جا', 'start': 37.156, 'end': 37.256, 'score': 0.25}, {'word': 'کے', 'start': 37.276, 'end': 37.357, 'score': 0.244}, {'word': 'فارم', 'start': 37.417, 'end': 37.578, 'score': 0.25}, {'word': 'فل', 'start': 37.618, 'end': 37.779, 'score': 0.23}, {'word': 'کریں', 'start': 37.799, 'end': 38.0, 'score': 0.368}]}, {'start': 38.0, 'end': 40.98, 'text': ' ٹھیک ہے میں ویب سائیٹ پر جا کے کرتا ہوں تینکیو', 'words': [{'word': 'ٹھیک', 'start': 38.0, 'end': 38.705, 'score': 0.404}, {'word': 'ہے', 'start': 38.725, 'end': 38.765, 'score': 0.492}, {'word': 'میں', 'start': 38.866, 'end': 39.107, 'score': 0.181}, {'word': 'ویب', 'start': 39.148, 'end': 39.289, 'score': 0.318}, {'word': 'سائیٹ', 'start': 39.329, 'end': 39.51, 'score': 0.312}, {'word': 'پر', 'start': 39.53, 'end': 39.591, 'score': 0.004}, {'word': 'جا', 'start': 39.631, 'end': 39.732, 'score': 0.414}, {'word': 'کے', 'start': 39.772, 'end': 39.872, 'score': 0.575}, {'word': 'کرتا', 'start': 39.893, 'end': 40.154, 'score': 0.47}, {'word': 'ہوں', 'start': 40.174, 'end': 40.235, 'score': 0.25}, {'word': 'تینکیو', 'start': 40.295, 'end': 40.98, 'score': 0.486}]}]

for i in range(len(want_data)):
    print(want_data[i])

{'start': 0.0, 'end': 7.0, 'text': ' السلام علیکم میں حبیبینک کا نمائندہ ہوں ہم آپ کو کارڈ لینے کی تجویز کریں گے کیا آپ انڈسٹر ہیں؟', 'words': [{'word': 'السلام', 'start': 0.0, 'end': 1.324, 'score': 0.164}, {'word': 'علیکم', 'start': 1.464, 'end': 1.825, 'score': 0.262}, {'word': 'میں', 'start': 1.926, 'end': 2.046, 'score': 0.403}, {'word': 'حبیبینک', 'start': 2.106, 'end': 2.668, 'score': 0.335}, {'word': 'کا', 'start': 2.688, 'end': 2.768, 'score': 0.603}, {'word': 'نمائندہ', 'start': 2.808, 'end': 3.269, 'score': 0.233}, {'word': 'ہوں', 'start': 3.289, 'end': 3.47, 'score': 0.083}, {'word': 'ہم', 'start': 3.59, 'end': 3.731, 'score': 0.567}, {'word': 'آپ', 'start': 3.751, 'end': 3.851, 'score': 0.006}, {'word': 'کو', 'start': 3.891, 'end': 3.971, 'score': 0.254}, {'word': 'کارڈ', 'start': 4.032, 'end': 4.252, 'score': 0.064}, {'word': 'لینے', 'start': 4.272, 'end': 4.473, 'score': 0.203}, {'word': 'کی', 'start': 4.493, 'end': 4.573, 'score': 0.279}, {'word': 'تجویز', 'start': 4.61

In [21]:
for i in range(len(want_data)):
    print(want_data[i]['words'])

[{'word': 'السلام', 'start': 0.0, 'end': 1.324, 'score': 0.164}, {'word': 'علیکم', 'start': 1.464, 'end': 1.825, 'score': 0.262}, {'word': 'میں', 'start': 1.926, 'end': 2.046, 'score': 0.403}, {'word': 'حبیبینک', 'start': 2.106, 'end': 2.668, 'score': 0.335}, {'word': 'کا', 'start': 2.688, 'end': 2.768, 'score': 0.603}, {'word': 'نمائندہ', 'start': 2.808, 'end': 3.269, 'score': 0.233}, {'word': 'ہوں', 'start': 3.289, 'end': 3.47, 'score': 0.083}, {'word': 'ہم', 'start': 3.59, 'end': 3.731, 'score': 0.567}, {'word': 'آپ', 'start': 3.751, 'end': 3.851, 'score': 0.006}, {'word': 'کو', 'start': 3.891, 'end': 3.971, 'score': 0.254}, {'word': 'کارڈ', 'start': 4.032, 'end': 4.252, 'score': 0.064}, {'word': 'لینے', 'start': 4.272, 'end': 4.473, 'score': 0.203}, {'word': 'کی', 'start': 4.493, 'end': 4.573, 'score': 0.279}, {'word': 'تجویز', 'start': 4.613, 'end': 5.074, 'score': 0.16}, {'word': 'کریں', 'start': 5.095, 'end': 5.275, 'score': 0.453}, {'word': 'گے', 'start': 5.315, 'end': 5.436, '